In [1]:
# import packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
import datetime as dt
from sklearn import linear_model
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

# suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
###Visualize the Map
import imageio
import folium
import folium.plugins as plugins
#from mpl_toolkits.basemap import Basemap

In [6]:
#First Take a look at the dataset and Its Size

In [7]:
business = pd.read_csv("yelp_business.csv")
business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [8]:
hours = pd.read_csv("yelp_business_hours.csv")
hours.head()

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
1,He-G7vWjzVUysIKrfNbPUQ,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
2,KQPW8lFf1y5BT2MxiSZ3QA,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
4,PfOCPjBrlQAnz__NXj9h_w,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0


In [9]:
checkin = pd.read_csv("yelp_checkin.csv")
checkin.head()

,business_id,weekday,hour,checkins
0,3Mc-LxcqeguOXOVT_2ZtCg,Tue,0:00,12
1,SVFx6_epO22bZTZnKwlX7g,Wed,0:00,4
2,vW9aLivd4-IorAfStzsHww,Tue,14:00,1
3,tEzxhauTQddACyqdJ0OPEQ,Fri,19:00,1
4,CEyZU32P-vtMhgqRCaXzMA,Tue,17:00,1


In [10]:
tip = pd.read_csv("yelp_tip.csv")
tip.head()

,text,date,likes,business_id,user_id
0,Great breakfast large portions and friendly wa...,2015-08-12,0,jH19V2I9fIslnNhDzPmdkA,ZcLKXikTHYOnYt5VYRO5sg
1,Nice place. Great staff. A fixture in the tow...,2014-06-20,0,dAa0hB2yrnHzVmsCkN4YvQ,oaYhjqBbh18ZhU0bpyzSuw
2,Happy hour 5-7 Monday - Friday,2016-10-12,0,dAa0hB2yrnHzVmsCkN4YvQ,ulQ8Nyj7jCUR8M83SUMoRQ
3,"Parking is a premium, keep circling, you will ...",2017-01-28,0,ESzO3Av0b1_TzKOiqzbQYQ,ulQ8Nyj7jCUR8M83SUMoRQ
4,Homemade pasta is the best in the area,2017-02-25,0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ


In [2]:
review = pd.read_csv("yelp_review.csv")
review.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [12]:
user = pd.read_csv("yelp_user.csv")
user.head()

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg...",0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,None,4,0,0,0,None,...,0,0,0,0,0,0,0,0,1,0


In [13]:
business['city'].value_counts().head()

Las Vegas     26775
Phoenix       17213
Toronto       17206
Charlotte      8553
Scottsdale     8228
Name: city, dtype: int64

thus in here we pick LV as our city---of course if we use new york , then we just replace the dataset

1. Firstly we can visulize the resteraunts on the map

In [14]:
# # Sample it down to only the North America region 
# lon_min, lon_max = -132.714844, -59.589844
# lat_min, lat_max = 13.976715,56.395664

# #create the selector
# idx_NA = (business["longitude"]>lon_min) &\
#             (business["longitude"]<lon_max) &\
#             (business["latitude"]>lat_min) &\
#             (business["latitude"]<lat_max)
# #apply the selector to subset
# NA_business=business[idx_NA]

# #initiate the figure
# plt.figure(figsize=(12,6))
# m2 = Basemap(projection='merc',
#              llcrnrlat=lat_min,
#              urcrnrlat=lat_max,
#              llcrnrlon=lon_min,
#              urcrnrlon=lon_max,
#              lat_ts=35,
#              resolution='i')

# m2.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
# m2.drawmapboundary(fill_color='#000000')                # black background
# m2.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders

# # Plot the data
# mxy = m2(NA_business["longitude"].tolist(), NA_business["latitude"].tolist())
# m2.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.05, zorder=5)

# plt.title("North America Region")
# # Sample it down to only the Eurozone + Britain :p 
# lon_min, lon_max = -8.613281,16.699219
# lat_min, lat_max = 40.488737,59.204064

# #create the selector
# idx_euro = (business["longitude"]>lon_min) &\
#             (business["longitude"]<lon_max) &\
#             (business["latitude"]>lat_min) &\
#             (business["latitude"]<lat_max)
# #apply the selector to subset
# euro_business=business[idx_euro]

# #initiate the figure
# plt.figure(figsize=(12,6))
# m3 = Basemap(projection='merc',
#              llcrnrlat=lat_min,
#              urcrnrlat=lat_max,
#              llcrnrlon=lon_min,
#              urcrnrlon=lon_max,
#              lat_ts=35,
#              resolution='i')

# m3.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
# m3.drawmapboundary(fill_color='#000000')                # black background
# m3.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders

# # Plot the data
# mxy = m3(euro_business["longitude"].tolist(), euro_business["latitude"].tolist())
# m3.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.05, zorder=5)

# plt.title("Europe Region")
# plt.show()

##Extract the data just for specific city_as we pick LV

In [15]:
business_vegas=business[business['city']=='Las Vegas']

In [16]:
business_vegas=business_vegas.reset_index(drop=True)

In [17]:
import re
business_vegas['restaurant']=business_vegas['categories'].str.contains('Restaurants',flags=re.IGNORECASE)

In [18]:
business_vegas_restaurant=business_vegas[business_vegas['restaurant']==True]
business_vegas_restaurant

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,restaurant
6,Pd52CjgyEU3Rb8co6QfTPw,"""Flight Deck Bar & Grill""",Southeast,"""6730 S Las Vegas Blvd""",Las Vegas,NV,89119,36.066914,-115.170848,4.0,13,1,Nightlife;Bars;Barbeque;Sports Bars;American (...,True
7,4srfPk1s8nlm1YusyDUbjg,"""Subway""",Southeast,"""6889 S Eastern Ave, Ste 101""",Las Vegas,NV,89119,36.064652,-115.118954,2.5,6,1,Fast Food;Restaurants;Sandwiches,True
8,n7V4cD-KqqE3OXk0irJTyA,"""GameWorks""",Southeast,"""6587 Las Vegas Blvd S, Ste 171""",Las Vegas,NV,89119,36.068259,-115.178877,3.0,349,1,Arcades;Arts & Entertainment;Gastropubs;Restau...,True
14,F0fEKpTk7gAmuSFI0KW1eQ,"""Cafe Mastrioni""",Spring Valley,"""4250 S Rainbow Blvd, Ste 1007""",Las Vegas,NV,89103,36.111057,-115.241688,1.5,3,0,Italian;Restaurants,True
19,Wpt0sFHcPtV5MO9He7yMKQ,"""McDonald's""",Eastside,"""3020 E Desert Inn Rd""",Las Vegas,NV,89121,36.130013,-115.109310,2.0,20,1,Restaurants;Fast Food;Burgers,True
23,W1Yr6c2XDx_RBjb6WsV-aQ,"""Divine Cafe at the Springs Preserve""",Westside,"""333 S Valley View Blvd""",Las Vegas,NV,89152,36.170706,-115.190288,4.0,140,1,Restaurants;Cafes;American (New);Bars;Nightlif...,True
27,vzx1WdVivFsaN4QYrez2rw,"""Subway""",NaN,"""5111 Boulder Hwy""",Las Vegas,NV,89122,36.112895,-115.062353,3.0,3,1,Sandwiches;Restaurants;Fast Food,True
31,Uy3_5nLo3sYkAuSX6mjdmg,"""Geebee's Bar & Grill""",Southeast,"""8560 Las Vegas Blvd S""",Las Vegas,NV,89123,36.034244,-115.171409,4.5,33,0,Restaurants;American (Traditional),True
43,dPxZI9lrKTl5dvFfnb1_Ig,"""Trattoria Italia""",Anthem,"""9905 S Eastern Ave, Ste 140""",Las Vegas,NV,89183,36.010086,-115.118656,4.5,210,1,Seafood;Italian;Pizza;Restaurants,True
47,WfB_SsYeKy83QQsqAAyGVQ,"""Cancun Bar & Grill""",Southeast,"""5006 S Maryland Pkwy, Ste 17""",Las Vegas,NV,89119,36.098632,-115.136079,4.5,5,1,Karaoke;Bars;Mexican;Restaurants;Nightlife;Dan...,True


In [ ]:
## As the index is a liite bit messy, so i just organized it
#index=business_vegas_restaurant.reset_index(drop=True).head()

In [19]:
business_vegas_restaurant2=business_vegas_restaurant.copy()

In [20]:
business_vegas_restaurant2 = business_vegas_restaurant2['categories'].str.split(';', expand=True)

In [21]:
business_vegas_restaurant2.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
6,Nightlife,Bars,Barbeque,Sports Bars,American (New),Restaurants,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,Fast Food,Restaurants,Sandwiches,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Arcades,Arts & Entertainment,Gastropubs,Restaurants,American (New),None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14,Italian,Restaurants,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19,Restaurants,Fast Food,Burgers,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [22]:
cate=business_vegas_restaurant2.iloc[:,:8]
cate.head()

,0,1,2,3,4,5,6,7
6,Nightlife,Bars,Barbeque,Sports Bars,American (New),Restaurants,None,None
7,Fast Food,Restaurants,Sandwiches,None,None,None,None,None
8,Arcades,Arts & Entertainment,Gastropubs,Restaurants,American (New),None,None,None
14,Italian,Restaurants,None,None,None,None,None,None
19,Restaurants,Fast Food,Burgers,None,None,None,None,None


In [23]:
#cate.rename(columns = {'0': 'Cate_0'},inplace=True)
#cate.head()

In [24]:
list(cate[0].unique())

['Nightlife',
 'Fast Food',
 'Arcades',
 'Italian',
 'Restaurants',
 'Sandwiches',
 'Seafood',
 'Karaoke',
 'Dance Clubs',
 'Burgers',
 'Food',
 'Delis',
 'Buffets',
 'Event Planning & Services',
 'American (New)',
 'Hawaiian',
 'Arts & Entertainment',
 'Salad',
 'Bars',
 'Pubs',
 'Chinese',
 'Pizza',
 'Hot Dogs',
 'Sushi Bars',
 'Filipino',
 'Cocktail Bars',
 'Desserts',
 'Donuts',
 'Asian Fusion',
 'Food Trucks',
 'Mexican',
 'Barbeque',
 'Vietnamese',
 'Breakfast & Brunch',
 'Thai',
 'Chicken Wings',
 'Do-It-Yourself Food',
 'Soup',
 'Hot Pot',
 'Juice Bars & Smoothies',
 'American (Traditional)',
 'Vegan',
 'Vegetarian',
 'Bakeries',
 'Mediterranean',
 'Beer',
 'Korean',
 'Shopping',
 'Noodles',
 'Lounges',
 'Street Vendors',
 'Specialty Food',
 'Sports Bars',
 'Japanese',
 'Cheesesteaks',
 'Cafes',
 'Tex-Mex',
 'Indian',
 'Comfort Food',
 'Latin American',
 'Farmers Market',
 'Caterers',
 'Steakhouses',
 'Ice Cream & Frozen Yogurt',
 'Szechuan',
 'Persian/Iranian',
 'Halal',
 'Smo

In [25]:
business_vegas_restaurant=pd.concat([business_vegas_restaurant,cate], axis=1)
business_vegas_restaurant.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,categories,restaurant,0,1,2,3,4,5,6,7
6,Pd52CjgyEU3Rb8co6QfTPw,"""Flight Deck Bar & Grill""",Southeast,"""6730 S Las Vegas Blvd""",Las Vegas,NV,89119,36.066914,-115.170848,4.0,...,Nightlife;Bars;Barbeque;Sports Bars;American (...,True,Nightlife,Bars,Barbeque,Sports Bars,American (New),Restaurants,None,None
7,4srfPk1s8nlm1YusyDUbjg,"""Subway""",Southeast,"""6889 S Eastern Ave, Ste 101""",Las Vegas,NV,89119,36.064652,-115.118954,2.5,...,Fast Food;Restaurants;Sandwiches,True,Fast Food,Restaurants,Sandwiches,None,None,None,None,None
8,n7V4cD-KqqE3OXk0irJTyA,"""GameWorks""",Southeast,"""6587 Las Vegas Blvd S, Ste 171""",Las Vegas,NV,89119,36.068259,-115.178877,3.0,...,Arcades;Arts & Entertainment;Gastropubs;Restau...,True,Arcades,Arts & Entertainment,Gastropubs,Restaurants,American (New),None,None,None
14,F0fEKpTk7gAmuSFI0KW1eQ,"""Cafe Mastrioni""",Spring Valley,"""4250 S Rainbow Blvd, Ste 1007""",Las Vegas,NV,89103,36.111057,-115.241688,1.5,...,Italian;Restaurants,True,Italian,Restaurants,None,None,None,None,None,None
19,Wpt0sFHcPtV5MO9He7yMKQ,"""McDonald's""",Eastside,"""3020 E Desert Inn Rd""",Las Vegas,NV,89121,36.130013,-115.109310,2.0,...,Restaurants;Fast Food;Burgers,True,Restaurants,Fast Food,Burgers,None,None,None,None,None


In [26]:
business_vegas_restaurant[0]

6                        Nightlife
7                        Fast Food
8                          Arcades
14                         Italian
19                     Restaurants
23                     Restaurants
27                      Sandwiches
31                     Restaurants
43                         Seafood
47                         Karaoke
55                     Restaurants
57                     Dance Clubs
61                         Burgers
65                     Restaurants
73                     Restaurants
84                     Restaurants
90                     Restaurants
94                            Food
96                       Nightlife
98                     Restaurants
102                          Delis
106                    Restaurants
107                        Burgers
111                    Restaurants
117                        Buffets
118      Event Planning & Services
123                 American (New)
125                       Hawaiian
128           Arts &

In [27]:
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder() 

In [28]:
for i in range(0,8):
    business_vegas_restaurant[i]= le.fit_transform(business_vegas_restaurant[i]) 

In [29]:
business_vegas_restaurant.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,categories,restaurant,0,1,2,3,4,5,6,7
6,Pd52CjgyEU3Rb8co6QfTPw,"""Flight Deck Bar & Grill""",Southeast,"""6730 S Las Vegas Blvd""",Las Vegas,NV,89119,36.066914,-115.170848,4.0,...,Nightlife;Bars;Barbeque;Sports Bars;American (...,True,134,19,16,173,7,135,0,0
7,4srfPk1s8nlm1YusyDUbjg,"""Subway""",Southeast,"""6889 S Eastern Ave, Ste 101""",Las Vegas,NV,89119,36.064652,-115.118954,2.5,...,Fast Food;Restaurants;Sandwiches,True,69,166,152,0,0,0,0,0
8,n7V4cD-KqqE3OXk0irJTyA,"""GameWorks""",Southeast,"""6587 Las Vegas Blvd S, Ste 171""",Las Vegas,NV,89119,36.068259,-115.178877,3.0,...,Arcades;Arts & Entertainment;Gastropubs;Restau...,True,11,12,80,156,7,0,0,0
14,F0fEKpTk7gAmuSFI0KW1eQ,"""Cafe Mastrioni""",Spring Valley,"""4250 S Rainbow Blvd, Ste 1007""",Las Vegas,NV,89103,36.111057,-115.241688,1.5,...,Italian;Restaurants,True,106,166,0,0,0,0,0,0
19,Wpt0sFHcPtV5MO9He7yMKQ,"""McDonald's""",Eastside,"""3020 E Desert Inn Rd""",Las Vegas,NV,89121,36.130013,-115.109310,2.0,...,Restaurants;Fast Food;Burgers,True,154,75,29,0,0,0,0,0


# NLP-Sentiment Analysis(Trail)

In [4]:
# i will import seperately later after that

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
import seaborn as sn  
import re  
from textblob import TextBlob 
import json
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.model_selection import GridSearchCV

In order to use the plot much more easier 

In [5]:
pd.options.display.max_colwidth = 60

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['legend.fontsize'] = 'x-large'
plt.rcParams['axes.labelsize'] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize'] = 'x-large'
plt.rcParams['ytick.labelsize'] = 'x-large'

sn.set_style('darkgrid')
sn.set_context('notebook', font_scale=1.5)

In [6]:
text1=review['text']
df=pd.DataFrame(text1)

In [7]:
df1=df.head()

In [8]:
df1['word_count']= df1['text'].apply(lambda x: len(x.split()))
df1.head()

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,word_count
0,Super simple place but amazing nonetheless. It's been ar...,35
1,Small unassuming place that changes their menu every so ...,91
2,Lester's is located in a beautiful neighborhood and has ...,67
3,Love coming here. Yes the place always needs the floor s...,101
4,Had their chocolate almond croissant and it was amazing!...,41


In [9]:
df1['char_count']= df1['text'].apply(lambda x: len(x))

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
def average_word(x):
    words=x.split()
    return sum(len(word) for word in words)/len(words)

In [11]:
df1['average_char_count']=df1['text'].apply(lambda x:average_word(x))

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1.head()

,text,word_count,char_count,average_char_count
0,Super simple place but amazing nonetheless. It's been ar...,35,213,5
1,Small unassuming place that changes their menu every so ...,91,502,4
2,Lester's is located in a beautiful neighborhood and has ...,67,373,4
3,Love coming here. Yes the place always needs the floor s...,101,523,4
4,Had their chocolate almond croissant and it was amazing!...,41,232,4


In [14]:
from nltk.corpus import stopwords

In [15]:
stop_words=stopwords.words('english')

In [16]:
stop_words

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u"you're",
 u"you've",
 u"you'll",
 u"you'd",
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u"she's",
 u'her',
 u'hers',
 u'herself',
 u'it',
 u"it's",
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u"that'll",
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'th

In [17]:
len(stop_words)

179

In [18]:
df1['stop_words_count']=df1['text'].apply(lambda x:len([word for word in x.split() if word.lower() in stop_words]))
df1.head()

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  """Entry point for launching an IPython kernel.
/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,word_count,char_count,average_char_count,stop_words_count
0,Super simple place but amazing nonetheless. It's been ar...,35,213,5,15
1,Small unassuming place that changes their menu every so ...,91,502,4,40
2,Lester's is located in a beautiful neighborhood and has ...,67,373,4,34
3,Love coming here. Yes the place always needs the floor s...,101,523,4,46
4,Had their chocolate almond croissant and it was amazing!...,41,232,4,22


In [19]:
df1['stopword_rate']=df1['stop_words_count']/df1['word_count']

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df1.sort_values(by='stopword_rate')

,text,word_count,char_count,average_char_count,stop_words_count,stopword_rate
0,Super simple place but amazing nonetheless. It's been ar...,35,213,5,15,0.428571
1,Small unassuming place that changes their menu every so ...,91,502,4,40,0.439560
3,Love coming here. Yes the place always needs the floor s...,101,523,4,46,0.455446
2,Lester's is located in a beautiful neighborhood and has ...,67,373,4,34,0.507463
4,Had their chocolate almond croissant and it was amazing!...,41,232,4,22,0.536585


In [21]:
df1.describe()

,word_count,char_count,average_char_count,stop_words_count,stopword_rate
count,5.000000,5.00000,5.000000,5.00000,5.000000
mean,67.000000,368.60000,4.200000,31.40000,0.473525
std,29.291637,145.36609,0.447214,12.75931,0.046448
min,35.000000,213.00000,4.000000,15.00000,0.428571
25%,41.000000,232.00000,4.000000,22.00000,0.439560
50%,67.000000,373.00000,4.000000,34.00000,0.455446
75%,91.000000,502.00000,4.000000,40.00000,0.507463
max,101.000000,523.00000,5.000000,46.00000,0.536585


In [22]:
df1['lowercase']=df1['text'].apply(lambda x: " ".join(word.lower() for word in x.split()))

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df1['lowercase']

0    super simple place but amazing nonetheless. it's been ar...
1    small unassuming place that changes their menu every so ...
2    lester's is located in a beautiful neighborhood and has ...
3    love coming here. yes the place always needs the floor s...
4    had their chocolate almond croissant and it was amazing!...
Name: lowercase, dtype: object

In [24]:
df1['punctuation']=df1['lowercase'].str.replace('[^\w\s]','')

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df1['stopwords']=df1['punctuation'].apply(lambda x: " ".join(word for word in x.split()if word not in stop_words))

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
df1.head()

,text,word_count,char_count,average_char_count,stop_words_count,stopword_rate,lowercase,punctuation,stopwords
0,Super simple place but amazing nonetheless. It's been ar...,35,213,5,15,0.428571,super simple place but amazing nonetheless. it's been ar...,super simple place but amazing nonetheless its been arou...,super simple place amazing nonetheless around since 30s ...
1,Small unassuming place that changes their menu every so ...,91,502,4,40,0.439560,small unassuming place that changes their menu every so ...,small unassuming place that changes their menu every so ...,small unassuming place changes menu every often cool dec...
2,Lester's is located in a beautiful neighborhood and has ...,67,373,4,34,0.507463,lester's is located in a beautiful neighborhood and has ...,lesters is located in a beautiful neighborhood and has b...,lesters located beautiful neighborhood since 1951 known ...
3,Love coming here. Yes the place always needs the floor s...,101,523,4,46,0.455446,love coming here. yes the place always needs the floor s...,love coming here yes the place always needs the floor sw...,love coming yes place always needs floor swept give pean...
4,Had their chocolate almond croissant and it was amazing!...,41,232,4,22,0.536585,had their chocolate almond croissant and it was amazing!...,had their chocolate almond croissant and it was amazing ...,chocolate almond croissant amazing light buttery oh choc...


In [27]:
pd.Series(" ".join(df1['stopwords']).split()).value_counts()[:30]

sandwich      3
always        3
amazing       3
bit           3
place         3
go            3
meat          3
fries         3
order         3
burger        2
patty         2
inside        2
getting       2
montreal      2
since         2
outside       2
light         2
lesters       2
small         2
less          2
smoked        2
spot          1
12            1
super         1
chocolaty     1
1951          1
old           1
known         1
unassuming    1
tourists      1
dtype: int64

In [28]:
other_stop_word=['place','burger','12','outside']

In [29]:
len(other_stop_word)

4

In [30]:
df1['clean']=df1['stopwords'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stop_word))

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
pd.Series(" ".join(df1['clean']).split()).value_counts()[:30]

always       3
bit          3
order        3
go           3
amazing      3
meat         3
sandwich     3
fries        3
lesters      2
since        2
getting      2
patty        2
small        2
inside       2
montreal     2
light        2
smoked       2
less         2
downtown     1
food         1
chocolate    1
super        1
chocolaty    1
1951         1
old          1
known        1
wont         1
tourists     1
minute       1
cool         1
dtype: int64

In [32]:
df1.head()

,text,word_count,char_count,average_char_count,stop_words_count,stopword_rate,lowercase,punctuation,stopwords,clean
0,Super simple place but amazing nonetheless. It's been ar...,35,213,5,15,0.428571,super simple place but amazing nonetheless. it's been ar...,super simple place but amazing nonetheless its been arou...,super simple place amazing nonetheless around since 30s ...,super simple amazing nonetheless around since 30s still ...
1,Small unassuming place that changes their menu every so ...,91,502,4,40,0.439560,small unassuming place that changes their menu every so ...,small unassuming place that changes their menu every so ...,small unassuming place changes menu every often cool dec...,small unassuming changes menu every often cool decor vib...
2,Lester's is located in a beautiful neighborhood and has ...,67,373,4,34,0.507463,lester's is located in a beautiful neighborhood and has ...,lesters is located in a beautiful neighborhood and has b...,lesters located beautiful neighborhood since 1951 known ...,lesters located beautiful neighborhood since 1951 known ...
3,Love coming here. Yes the place always needs the floor s...,101,523,4,46,0.455446,love coming here. yes the place always needs the floor s...,love coming here yes the place always needs the floor sw...,love coming yes place always needs floor swept give pean...,love coming yes always needs floor swept give peanuts sh...
4,Had their chocolate almond croissant and it was amazing!...,41,232,4,22,0.536585,had their chocolate almond croissant and it was amazing!...,had their chocolate almond croissant and it was amazing ...,chocolate almond croissant amazing light buttery oh choc...,chocolate almond croissant amazing light buttery oh choc...


In [33]:
### Limitazation #condense what the msg is / done-do

In [34]:
#stemming # because--cause which is worng

### Lemmatization

In [35]:
#import Textbolob
from textblob import Word

In [39]:
df1['clean']

0    super simple amazing nonetheless around since 30s still ...
1    small unassuming changes menu every often cool decor vib...
2    lesters located beautiful neighborhood since 1951 known ...
3    love coming yes always needs floor swept give peanuts sh...
4    chocolate almond croissant amazing light buttery oh choc...
Name: clean, dtype: object

In [ ]:
# in here most of the time you will meet a package prob ;（ 哭哭了（倒入+测试）

In [46]:
# import nltk
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [47]:
# from nltk.corpus import brown
# brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

In [48]:
df1['lemmatize']=df1['clean'].apply(lambda x:" ".join(Word(word).lemmatize() for word in x.split()))

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Sentiment Analysis

In [51]:
from textblob import TextBlob

In [ ]:
## in here u can see the matrix first is polilarity and second is subjectivity

In [55]:
df1['polarity']=df1['lemmatize'].apply(lambda x: TextBlob(x).sentiment[0])## just got polarity

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
df1['subjectivity']=df1['lemmatize'].apply(lambda x: TextBlob(x).sentiment[1])## just got polarity

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
df1.describe()

,word_count,char_count,average_char_count,stop_words_count,stopword_rate,subjectivity,polarity
count,5.000000,5.00000,5.000000,5.00000,5.000000,5.000000,5.000000
mean,67.000000,368.60000,4.200000,31.40000,0.473525,0.689052,0.410000
std,29.291637,145.36609,0.447214,12.75931,0.046448,0.184252,0.305560
min,35.000000,213.00000,4.000000,15.00000,0.428571,0.527404,0.006250
25%,41.000000,232.00000,4.000000,22.00000,0.439560,0.605952,0.327083
50%,67.000000,373.00000,4.000000,34.00000,0.455446,0.611905,0.366667
75%,91.000000,502.00000,4.000000,40.00000,0.507463,0.700000,0.500000
max,101.000000,523.00000,5.000000,46.00000,0.536585,1.000000,0.850000


In [57]:
df1.head()

,text,word_count,char_count,average_char_count,stop_words_count,stopword_rate,lowercase,punctuation,stopwords,clean,lemmatize,subjectivity,polarity
0,Super simple place but amazing nonetheless. It's been ar...,35,213,5,15,0.428571,super simple place but amazing nonetheless. it's been ar...,super simple place but amazing nonetheless its been arou...,super simple place amazing nonetheless around since 30s ...,super simple amazing nonetheless around since 30s still ...,super simple amazing nonetheless around since 30 still s...,0.605952,0.327083
1,Small unassuming place that changes their menu every so ...,91,502,4,40,0.439560,small unassuming place that changes their menu every so ...,small unassuming place that changes their menu every so ...,small unassuming place changes menu every often cool dec...,small unassuming changes menu every often cool decor vib...,small unassuming change menu every often cool decor vibe...,0.611905,0.366667
2,Lester's is located in a beautiful neighborhood and has ...,67,373,4,34,0.507463,lester's is located in a beautiful neighborhood and has ...,lesters is located in a beautiful neighborhood and has b...,lesters located beautiful neighborhood since 1951 known ...,lesters located beautiful neighborhood since 1951 known ...,lesters located beautiful neighborhood since 1951 known ...,1.000000,0.850000
3,Love coming here. Yes the place always needs the floor s...,101,523,4,46,0.455446,love coming here. yes the place always needs the floor s...,love coming here yes the place always needs the floor sw...,love coming yes place always needs floor swept give pean...,love coming yes always needs floor swept give peanuts sh...,love coming yes always need floor swept give peanut shel...,0.527404,0.006250
4,Had their chocolate almond croissant and it was amazing!...,41,232,4,22,0.536585,had their chocolate almond croissant and it was amazing!...,had their chocolate almond croissant and it was amazing ...,chocolate almond croissant amazing light buttery oh choc...,chocolate almond croissant amazing light buttery oh choc...,chocolate almond croissant amazing light buttery oh choc...,0.700000,0.500000


In [ ]:
## cleaned the data that what we don't need

In [59]:
df1.drop(['lowercase','punctuation','stopwords','clean','lemmatize','average_char_count'],axis=1, inplace=True)

/Users/cindy/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [61]:
df1.sort_values(by='polarity')

,text,word_count,char_count,stop_words_count,stopword_rate,subjectivity,polarity
3,Love coming here. Yes the place always needs the floor s...,101,523,46,0.455446,0.527404,0.006250
0,Super simple place but amazing nonetheless. It's been ar...,35,213,15,0.428571,0.605952,0.327083
1,Small unassuming place that changes their menu every so ...,91,502,40,0.439560,0.611905,0.366667
4,Had their chocolate almond croissant and it was amazing!...,41,232,22,0.536585,0.700000,0.500000
2,Lester's is located in a beautiful neighborhood and has ...,67,373,34,0.507463,1.000000,0.850000
